In [1]:
!pip install requests beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

class WikipediaMovieScraper:
    BASE_URL = "https://en.wikipedia.org"
    
    def __init__(self, list_url):
        self.list_url = list_url
        self.films_data = []
    
    def _get_html(self, url):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            return response.text
        except (requests.RequestException, ConnectionError) as e:
            print(f"Error fetching {url}: {e}")
            return None
    
    def _parse_table_rows(self, html):
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find('table', {'class': 'wikitable'})
        return table.find_all('tr') if table else []
    
    def get_movie_list(self):
        html = self._get_html(self.list_url)
        if not html:
            return []
        return self._parse_table_rows(html)
    
    def _extract_image_url(self, table):
        img_td = table.find('td', class_='infobox-image')
        if not img_td:
            return None
        
        img_tag = img_td.find('img')
        if not img_tag or 'src' not in img_tag.attrs:
            return None
        
        src = img_tag['src']
        if src.startswith('//'):
            src = f'https:{src}'
        return urljoin(self.BASE_URL, src)
    
    def _extract_director(self, data):
        director_links = data.find_all('a')
        if director_links:
            # If there's only one link, check if the text contains concatenated names
            if len(director_links) == 1:
                director_text = director_links[0].get_text(strip=True)
                # Split when a lowercase letter is immediately followed by an uppercase letter
                directors = re.split(r'(?<=[a-z])(?=[A-Z])', director_text)
                return [d.strip() for d in directors if d.strip()]
            else:
                return [link.get_text(strip=True) for link in director_links]
        # Fallback if no <a> tags are found
        director_text = data.get_text(strip=True)
        directors = re.split(r'(?<=[a-z])(?=[A-Z])', director_text)
        return [d.strip() for d in directors if d.strip()]
    
    def _clean_box_office(self, text):
        cleaned = text.replace('\xa0', ' ').replace('\u202f', ' ').strip()
        # Insert space between number and unit if missing
        cleaned = re.sub(
            r'(\d+(?:\.\d+)?)([bm]illion)',
            r'\1 \2',
            cleaned,
            flags=re.IGNORECASE
        )
        return cleaned
    
    def _extract_country(self, data):
        country_elements = data.find_all(['li', 'a'])
        if country_elements:
            return [elem.get_text(strip=True) for elem in country_elements]
        
        text = data.get_text(separator=', ', strip=True)
        text = re.sub(r'(?<!^)(United)(?=\s)', r', \1', text)  # Fix concatenated countries
        return [c.strip() for c in text.split(',') if c.strip()]
    
    def get_movie_details(self, url):
        html = self._get_html(url)
        if not html:
            return {}, None
        
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find('table', {'class': 'infobox vevent'})
        if not table:
            return {}, None
        
        details = {
            'director': [],
            'box_office': '',
            'country': [],
        }
        
        # Remove reference superscripts first
        for sup in table.find_all("sup"):
            sup.decompose()
        
        for row in table.find_all('tr'):
            header = row.find('th')
            data = row.find('td')
            if not header or not data:
                continue
            
            header_text = header.get_text(strip=True)
            if header_text == "Directed by":
                details['director'] = self._extract_director(data)
            elif header_text == "Box office":
                details['box_office'] = self._clean_box_office(data.get_text(strip=True))
            elif header_text in {"Country", "Country of origin", "Countries"}:
                details['country'] = self._extract_country(data)
        
        return details, self._extract_image_url(table)
    
    def _parse_movie_row(self, row):
        cols = row.find_all(['th', 'td'])
        
        title_col = cols[2]
        year_col = cols[4]
        rank = int(cols[0].get_text(strip=True))
        
        link_tag = title_col.find('a')
        if not link_tag or 'href' not in link_tag.attrs:
            return None
        
        details_url = urljoin(self.BASE_URL, link_tag['href'])
        details, img_url = self.get_movie_details(details_url)
        
        try:
            year_text = year_col.get_text(strip=True)
            release_year = int(re.search(r'\d{4}', year_text).group()) if year_text else None
        except (ValueError, AttributeError):
            release_year = None
        
        return {
            'title': title_col.get_text(strip=True).replace("†", "").strip(),
            'release_year': release_year,
            'director': details['director'],
            'box_office': details['box_office'][details['box_office'].find('$'):],
            'country': details['country'],
            'img_url': img_url,
            'rank': rank
        }
    
    def scrape(self):
        rows = self.get_movie_list()
        for row in rows[1:]:  # Skip header
            film_data = self._parse_movie_row(row)
            if film_data:
                self.films_data.append(film_data)
        return self.films_data


LIST_URL = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"
scraper = WikipediaMovieScraper(LIST_URL)
films_data = scraper.scrape()
print(f"Scraped {len(films_data)} movies")

films_data

Scraped 50 movies


[{'title': 'Avatar',
  'release_year': 2009,
  'director': ['James Cameron'],
  'box_office': '$2.923 billion',
  'country': ['United Kingdom', 'United States'],
  'img_url': 'https://upload.wikimedia.org/wikipedia/en/thumb/d/d6/Avatar_%282009_film%29_poster.jpg/220px-Avatar_%282009_film%29_poster.jpg',
  'rank': 1},
 {'title': 'Avengers: Endgame',
  'release_year': 2019,
  'director': ['Anthony Russo', 'Joe Russo'],
  'box_office': '$2.799 billion',
  'country': ['United States'],
  'img_url': 'https://upload.wikimedia.org/wikipedia/en/0/0d/Avengers_Endgame_poster.jpg',
  'rank': 2},
 {'title': 'Avatar: The Way of Water',
  'release_year': 2022,
  'director': ['James Cameron'],
  'box_office': '$2.320 billion',
  'country': ['United States'],
  'img_url': 'https://upload.wikimedia.org/wikipedia/en/thumb/5/54/Avatar_The_Way_of_Water_poster.jpg/220px-Avatar_The_Way_of_Water_poster.jpg',
  'rank': 3},
 {'title': 'Titanic',
  'release_year': 1997,
  'director': ['James Cameron'],
  'box_o

In [3]:
import sqlite3
import json

# Connect to the database (creates films.db if it doesn't exist)
conn = sqlite3.connect('films.db')
cursor = conn.cursor()

# Create the movies table with UNIQUE constraints (if not exists)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS movies (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        release_year INTEGER,
        director TEXT,
        box_office TEXT,
        country TEXT,
        img_url TEXT,
        rank INTEGER UNIQUE,
        UNIQUE(title, release_year)
    )
''')

# Insert data into the table using INSERT OR IGNORE to skip duplicates
for film in films_data:
    cursor.execute('''
        INSERT OR IGNORE INTO movies (title, release_year, director, box_office, country, img_url, rank)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        film['title'],
        film['release_year'],
        json.dumps(film['director']),  # Convert list to JSON string
        film['box_office'],
        json.dumps(film['country']),     # Convert list to JSON string
        film['img_url'],
        film['rank']
    ))

conn.commit()

# Optional: Retrieve and print all rows to verify insertion
cursor.execute("SELECT * FROM movies")
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

(1, 'Avatar', 2009, '["James Cameron"]', '$2.923 billion', '["United Kingdom", "United States"]', 'https://upload.wikimedia.org/wikipedia/en/thumb/d/d6/Avatar_%282009_film%29_poster.jpg/220px-Avatar_%282009_film%29_poster.jpg', 1)
(2, 'Avengers: Endgame', 2019, '["Anthony Russo", "Joe Russo"]', '$2.799 billion', '["United States"]', 'https://upload.wikimedia.org/wikipedia/en/0/0d/Avengers_Endgame_poster.jpg', 2)
(3, 'Avatar: The Way of Water', 2022, '["James Cameron"]', '$2.320 billion', '["United States"]', 'https://upload.wikimedia.org/wikipedia/en/thumb/5/54/Avatar_The_Way_of_Water_poster.jpg/220px-Avatar_The_Way_of_Water_poster.jpg', 3)
(4, 'Titanic', 1997, '["James Cameron"]', '$2.264 billion', '["United States"]', 'https://upload.wikimedia.org/wikipedia/en/1/18/Titanic_%281997_film%29_poster.png', 4)
(5, 'Star Wars: The Force Awakens', 2015, '["J. J. Abrams"]', '$2.07 billion', '["United States"]', 'https://upload.wikimedia.org/wikipedia/en/a/a2/Star_Wars_The_Force_Awakens_Theatr

In [4]:
import sqlite3
import json

def export_to_json(db_path, json_file_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Retrieve all rows from the movies table
    cursor.execute("SELECT * FROM movies")
    rows = cursor.fetchall()
    
    films = []
    for row in rows:
        film = {
            "id": row[0],
            "title": row[1],
            "release_year": row[2],
            # Stored as JSON strings in the DB; convert them back to lists
            "director": json.loads(row[3]),
            "box_office": row[4],
            "country": json.loads(row[5]),
            "img_url": row[6],
            "rank": row[7]
        }
        films.append(film)
    
    conn.close()
    
    # Write the list of films to a JSON file with indentation for readability
    with open(json_file_path, 'w') as f:
        json.dump(films, f, indent=4)
    
    print(f"Database content exported to {json_file_path}")

export_to_json('films.db', 'films_data.json')

Database content exported to films_data.json
